# Rook und Ceph

**Ceph** ist ein verteiltes, fehlertolerantes und hochskalierbares Speichersystem, das Objektspeicher, Blockspeicher (RBD) und verteilte Dateisysteme (CephFS) bereitstellt. Es nutzt das **CRUSH-Algorithmus**, um Daten effizient und ohne zentralen Flaschenhals auf Cluster-Knoten zu verteilen. Ceph wird häufig in Cloud- und Virtualisierungsumgebungen wie OpenStack oder Kubernetes eingesetzt.  

**Rook** ist ein **Cloud-Native Storage Orchestrator**, der Ceph in Kubernetes-Clustern automatisiert bereitstellt, verwaltet und überwacht. Rook vereinfacht die Bereitstellung und Verwaltung von Ceph, indem es als Operator in Kubernetes fungiert und automatisch Storage-Pools, Monitore und OSDs (Object Storage Daemons) verwaltet.  

**Vorteile von Rook und Ceph:**  
- **Skalierbarer, verteilter Speicher** für Block-, Datei- und Objektdaten  
- **Automatisches Self-Healing** und Fehlertoleranz  
- **Nahtlose Kubernetes-Integration** für Stateful-Workloads  
- **Open-Source & Software-Defined Storage**, unabhängig von proprietärer Hardware  

### rbd Modul

Das **rbd-Modul** des Linux-Kernels ist ein Treiber für das **RADOS Block Device (RBD)**, das in **Ceph**-Clustern verwendet wird. Es ermöglicht den Zugriff auf verteilte, skalierbare Speichergeräte, die über das Ceph-Cluster-Storage-System bereitgestellt werden. Das rbd-Modul stellt diese Speicher als reguläre Blockgeräte im System zur Verfügung, sodass sie wie lokale Festplatten verwendet werden können.  

Zu den Hauptfunktionen gehören:  
- Direkte Anbindung an Ceph-Cluster über das Netzwerk  
- Unterstützung für Snapshots und Klone  
- Dynamische Skalierung und hohe Verfügbarkeit  
- Effiziente Lastverteilung durch verteilten Speicher  

Das rbd-Modul wird typischerweise in Cloud-Umgebungen und Virtualisierungslösungen wie **OpenStack** oder **Kubernetes** genutzt, um hochverfügbaren und redundanten Speicher bereitzustellen.

---

Bevor wir rook-ceph enablen, muss rdb im Linux Kernel aktiviert sein:

In [1]:
%%bash
sudo modprobe rbd
lsmod | grep rbd

rbd                   126976  0
libceph               544768  1 rbd


Dann können wir rook-ceph enablen

In [3]:
%%bash
sudo microk8s enable rook-ceph

Infer repository core for addon rook-ceph


Add Rook Helm repository https://charts.rook.io/release
"rook-release" has been added to your repositories
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "rook-release" chart repository
Update Complete. ⎈Happy Helming!⎈
Install Rook version v1.11.9
NAME: rook-ceph
LAST DEPLOYED: Fri Feb  7 16:03:36 2025
NAMESPACE: rook-ceph
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
The Rook Operator has been installed. Check its status by running:
  kubectl --namespace rook-ceph get pods -l "app=rook-ceph-operator"

Visit https://rook.io/docs/rook/latest for instructions on how to create and configure Rook clusters

Important Notes:
- You must customize the 'CephCluster' resource in the sample manifests for your cluster.
- Each CephCluster must be deployed to its own namespace, the samples use `rook-ceph` for the namespace.
- The sample manifests assume you also installed the rook-ceph operator in the `rook-ceph` namespace.
- The he

In [4]:
! kubectl get pods -n rook-ceph

NAME                                  READY   STATUS    RESTARTS   AGE
rook-ceph-operator-6fcdbf47f9-7lq4w   1/1     Running   0          25s


### Host Storage Cluster

Um rook-ceph auszuprobieren, verwenden wir die Variante "Host Storage Cluster".

In einem "Host Storage Cluster" konfiguriert Rook Ceph so, dass Daten direkt auf dem Host gespeichert werden. 

In [7]:
%%bash
kubectl apply -f - <<EOF
apiVersion: ceph.rook.io/v1
kind: CephCluster
metadata:
  name: rook-ceph
  namespace: rook-ceph
spec:
  cephVersion:
    # see the "Cluster Settings" section below for more details on which image of ceph to run
    image: quay.io/ceph/ceph:v17.2.6
  dataDirHostPath: /var/lib/rook
  mon:
    count: 3
    allowMultiplePerNode: true
  storage:
    useAllNodes: true
    useAllDevices: true
EOF


cephcluster.ceph.rook.io/rook-ceph configured


### Links

* [Host Storage Cluster](https://rook.io/docs/rook/v1.11/CRDs/Cluster/host-cluster/)